In [30]:
import numpy as np
import pandas as pd
import torch
import random
import matplotlib.cm as cmx
from mpl_toolkits.mplot3d import Axes3D
import math
import sys
from mpl_toolkits.mplot3d import Axes3D  # registers 3D projection, not used directly
import matplotlib.pyplot as plt

def compare(x,y):
    Flag = True
    if ((x[0] == y[0]) & (x[1] == y[1]) & (x[2] == y[2])):
        Flag = False
    return Flag

def dist_bt_points(x,y):
    x1,y1,z1 = x
    x2,y2,z2 = y
    return math.sqrt(pow((x1-x2),2) + pow((y1-y2),2) + pow((z1-z2),2))

def find_center(sample):
    x_sum = 0
    y_sum = 0
    z_sum = 0
    for i in sample:
        x_sum += i[0]
        y_sum += i[1]
        z_sum += i[2]
    return [x_sum/(sample.shape[0]+1), y_sum/(sample.shape[0]+1), z_sum/(sample.shape[0]+1)]

def dist2center(x,y,z,mid):
    return math.sqrt(pow(mid[0]-x,2) + pow(mid[0]-y,2) + pow(mid[0]-z,2))

def angle_bt_planes(plane_1,plane_2):
    a1,b1,c1,d1 = plane_1
    a2,b2,c2,d2 = plane_2
    return abs((a1*a2+b1*b2+c1*c2) / (math.sqrt(a1*a1+b1*b1+c1*c1) * math.sqrt(a2*a2+b2*b2+c2*c2)))
    

def create_plane(p1,p2,p3):
    
    # These two vectors are in the plane
    v1 = p3 - p1
    v2 = p2 - p1
    
    # the cross product is a vector normal to the plane
    cp = np.cross(v1, v2)
    a, b, c = cp

    # This evaluates a * x3 + b * y3 + c * z3 which equals d
    d = np.dot(cp, p3)

    #print('The equation is {0}x + {1}y + {2}z = {3}'.format(a, b, c, d))
    return [a,b,c,d]

def dist2plane(planee, point):
    a,b,c,d = planee
    x1,y1,z1 = point
    return abs(a*x1+b*y1+c*z1-d) / math.sqrt(a*a+b*b+c*c)

from matplotlib.pyplot import savefig

def scatter3dd(x,y,z, ang1, ang2):
    
    fig = plt.figure(figsize=plt.figaspect(1)*1.5)
    ax = fig.gca(projection='3d')
    ax.scatter(x, y, z)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.view_init(ang1, ang2)
    plt.show()
    
def plane_intersect(a, b):
    
    a_vec, b_vec = np.array(a[:3]), np.array(b[:3])

    aXb_vec = np.cross(a_vec, b_vec)

    A = np.array([a_vec, b_vec, aXb_vec])
    d = np.array([-a[3], -b[3], 0.]).reshape(3,1)

    p_inter = np.linalg.solve(A, d).T
    
    dot1 = p_inter[0]
    dot2 = (p_inter + aXb_vec)[0]
    
    
    return [[-dot1[0], -dot1[1], -dot1[2]], [dot2[0]-dot1[0], dot2[1]-dot1[1], dot2[2]-dot1[2]]]
    # x+x1/p1 = y+y1/p2 = z+z1/p3
    # return [x1,y1,z1],[p1,p2,p3]
    
def dist_bt_lines(a,b):
    x = a[1]
    y = b[1]
    z = np.cross(x, y)
    q = [-a[0][0] + b[0][0], -a[0][1] + b[0][1],-a[0][2] + b[0][2]]
    count1 = np.linalg.det([x,y,q])
    count2 = 0
    for i in z:
        count2 += i*i
    return abs(count1)/math.sqrt(count2)

def check_equals_planes(plane_1, plane_2):
    total = []
    for i in range(4):
        total.append((plane_1[i]/plane_2[i]))
    if (abs(total[0] - total[1])) < 1e-5 and (abs(total[1] - total[2]) < 1e-5) and (abs(total[2] - total[3]) < 1e-5):
        return True
    else:
        return False
    
def build_paral_plane(pplane, side, center):
    new_plane = pplane[0:3]
    summ = 0
    for i in range(3):
        summ += new_plane[i]*new_plane[i]
    planev1 = new_plane.copy()
    planev2 = new_plane.copy()
    planev1.append(side * math.sqrt(summ)+pplane[3])
    planev2.append(-side*math.sqrt(summ) + pplane[3])
    if dist2plane(planev1,center) > dist2plane(planev2,center):
        return planev2
    else:
        return planev1
    
def find_cube_points(sample):
    
    
    mean_x = sample['x'].max() + sample['x'].min()
    mean_y = sample['y'].max() + sample['y'].min()
    mean_z = sample['z'].max() + sample['z'].min()

    box1 = sample.loc[(sample['x'] < mean_x) & (sample['y'] < mean_y) & (sample['z'] < mean_z)]
    box2 = sample.loc[(sample['x'] < mean_x) & (sample['y'] < mean_y) & (sample['z'] > mean_z)]
    box3 = sample.loc[(sample['x'] < mean_x) & (sample['y'] > mean_y) & (sample['z'] < mean_z)]
    box4 = sample.loc[(sample['x'] < mean_x) & (sample['y'] > mean_y) & (sample['z'] > mean_z)]
    box5 = sample.loc[(sample['x'] > mean_x) & (sample['y'] < mean_y) & (sample['z'] < mean_z)]
    box6 = sample.loc[(sample['x'] > mean_x) & (sample['y'] < mean_y) & (sample['z'] > mean_z)]
    box7 = sample.loc[(sample['x'] > mean_x) & (sample['y'] > mean_y) & (sample['z'] < mean_z)]
    box8 = sample.loc[(sample['x'] > mean_x) & (sample['y'] > mean_y) & (sample['z'] > mean_z)]

    max_num = 0
    boxes = [box1,box2,box3,box4,box5,box6,box7,box8]
    for box in boxes:
        if box.shape[0] > max_num:
            max_num = box.shape[0]
            main_box = box

    max_num = 0
    box_range = []
    for i in range(int(30-max_side*10)):
        for j in range(int(30-max_side*10)):
            for k in range(int(30-max_side*10)):
                box = sample.loc[(sample['x'] > -1.5 + i*0.1) & (sample['x'] < -1.5 + i*0.1 + max_side) &
                                  (sample['y'] > -1.5 + j*0.1) & (sample['z'] < -1.5 + j*0.1 + max_side) &
                                  (sample['z'] > -1.5 + k*0.1) & (sample['z'] < -1.5 + k*0.1 + max_side)]
                if (box.shape[0] > max_num):
                    max_num = box.shape[0]
                    box_range = [i/10 - 1.5,j/10 - 1.5,k/10 - 1.5]
                    main_box = box
    return main_box
    
def half_search(main_box):
    min_x = main_box['x'].min()
    min_y = main_box['y'].min()
    min_z = main_box['z'].min()
    max_x = main_box['x'].max()
    max_y = main_box['y'].max()
    max_z = main_box['z'].max()
    new_sample = main_box
    box1 = new_sample.loc[(new_sample['x'] < (max_x + min_x)/2)]
    box2 = new_sample.loc[(new_sample['x'] > (max_x + min_x)/2)]
    box3 = new_sample.loc[(new_sample['y'] < (max_y + min_y)/2)]
    box4 = new_sample.loc[(new_sample['y'] > (max_y + min_y)/2)]
    box5 = new_sample.loc[(new_sample['z'] < (max_z + min_z)/2)]
    box6 = new_sample.loc[(new_sample['z'] > (max_z + min_z)/2)]
    max_num = 0
    boxes = [box1,box2,box3,box4,box5,box6]
    for box in boxes:
        if box.shape[0] > max_num:
            max_num = box.shape[0]
            main_box = box
    return main_box

def find_plane(main_box):

    sample =  main_box
    sample = sample.to_numpy()

    import math
    bestSupport = 0
    bestStd = 10000000
    best_dist = 10000000
    i = 0
    best_count = 0
    num_of_points = sample.shape[0]+1
    forseeable_support = math.floor(500/18)
    eps = 1 - forseeable_support/num_of_points
    alpha = 0.95
    for i in sample:
        for j in sample:
            for k in sample:
                if compare(i,j) & compare(i,k) & compare(j,k):
                    Plane = create_plane(i,j,k)
                    count = 0
                    points_dist = np.array([])
                    lst = np.array([])
                    random_pos = np.random.randint(num_of_points-1, size=3)

                    for i in sample:
                        if (dist2plane(Plane,i) < 1e-20):
                            points_dist = np.append(points_dist,dist2plane(Plane,i))
                    st = np.std(points_dist)
                    if (len(points_dist) > bestSupport):
                        bestSupport = len(points_dist)
                        best_points = points_dist
                        bestPlane = Plane 
                        bestStd = st
                        best_count = count
    return Plane

def delete_plane_dots(main_box, plane, decay):
    main_box_arr = main_box.to_numpy()
    main_box['class'] = 0
    for i in range(main_box.shape[0]):
        if (abs(dist2plane(plane, main_box_arr[i])) < max_side * decay):
            main_box['class'][i] = 1

    main_box = main_box.loc[main_box['class'] == 0]
    main_box = main_box.drop(['class'], axis = 1)
    
    return main_box

def find_1st_plane(sample, answer, points_num):
    main_box = find_cube_points(sample)
    center = find_center(main_box.to_numpy())

    while (main_box.shape[0] > points_num):
        main_box = half_search(main_box)
        
    plane = find_plane(main_box)
    
    ans = answer.to_numpy()
    #for i in ans:
    #    print(dist2plane(plane,i))
    
    return plane, center

def find_2nd_plane(sample, answer, plane_1, points_num, decay):
    main_box = find_cube_points(sample)
    main_box = main_box.set_index(np.arange(0, len(main_box)))

    new_box = delete_plane_dots(main_box, plane_1, decay)
    main_box = new_box

    while (main_box.shape[0] > points_num):
        main_box = half_search(main_box)

    plane = find_plane(main_box)

    ans = answer.to_numpy()
        
    return plane

def find_3rd_plane(sample0, answer0, plane_1, plane_2, points_num , decay):
    main_box = find_cube_points(sample0)
    main_box = main_box.set_index(np.arange(0, len(main_box)))

    new_box = delete_plane_dots(main_box,plane_1, decay)
    main_box = new_box
    new_box = delete_plane_dots(main_box,plane_2, decay)
    main_box = new_box

    while (main_box.shape[0] > points_num):
        main_box = half_search(main_box)

    Plane = find_plane(main_box)

    ans = answer0.to_numpy()
        
    return Plane

def find_4th_plane(sample, answer, plane_1, plane_2, plane_3, points_num, decay):
    main_box = find_cube_points(sample)
    main_box = main_box.set_index(np.arange(0, len(main_box)))

    new_box = delete_plane_dots(main_box,plane_1,decay)
    main_box = new_box
    new_box = delete_plane_dots(main_box,plane_2,decay)
    main_box = new_box
    new_box = delete_plane_dots(main_box,plane_3,decay)
    main_box = new_box

    while (main_box.shape[0] > points_num):
        main_box = half_search(main_box)

    Plane = find_plane(main_box)

    ans = answer.to_numpy()
    
    return Plane

def find_5th_plane(sample, answer, plane_1, plane_2, plane_3, plane_4, points_num, decay):
    main_box = find_cube_points(sample)
    main_box = main_box.set_index(np.arange(0, len(main_box)))

    new_box = delete_plane_dots(main_box,plane_1,decay)
    main_box = new_box
    new_box = delete_plane_dots(main_box,plane_2,decay)
    main_box = new_box
    new_box = delete_plane_dots(main_box,plane_3,decay)
    main_box = new_box
    new_box = delete_plane_dots(main_box,plane_4,decay)
    main_box = new_box

    while (main_box.shape[0] > points_num):
        main_box = half_search(main_box)

    our_plane = find_plane(main_box)

    ans = answer.to_numpy()
        
    return our_plane

def check_3_planes_corr(plane_1, plane_2, plane_3):
    planes_angles = [angle_bt_planes(plane_1,plane_2), angle_bt_planes(plane_1,plane_3), angle_bt_planes(plane_2,plane_3)]
    paralel_planes_count = 0
    perpen_planes_count = 0
    planes_problems = []
    for i in range(3):
        if abs((planes_angles[i]-1.0))< 1e-10:
            paralel_planes_count += 1
        elif abs(planes_angles[i]) < 1e-10:
            perpen_planes_count += 1
        else:
            planes_problems.append(i)
    return planes_problems, paralel_planes_count, perpen_planes_count

def check_4_planes_corr(plane_1, plane_2, plane_3, plane_4):
    planes_angles = [angle_bt_planes(plane_1,plane_2), angle_bt_planes(plane_1,plane_3), angle_bt_planes(plane_1,plane_4), 
                     angle_bt_planes(plane_2,plane_3), angle_bt_planes(plane_2,plane_4), angle_bt_planes(plane_3,plane_4)]
    paralel_planes_count = 0
    perpen_planes_count = 0
    paral_index = 0
    planes_problems = []
    for i in range(6):
        if abs((planes_angles[i]-1.0))< 1e-10:
            paralel_planes_count += 1
            paral_index = i
        elif abs(planes_angles[i]) < 1e-10:
            perpen_planes_count += 1
        else:
            planes_problems.append(i)
    return planes_problems, paralel_planes_count, perpen_planes_count, paral_index

def fix_plane(plane_1,plane_2,points_num,decay, sample, answer): 
    Flag = False
    for j in range(int(decay*10),21,1):
        new_decay = j/10.0
        for i in range(points_num-3, 0, -3):
            new_plane = find_3rd_plane(sample, answer, plane_1, plane_2, i, new_decay)
            new_check,a,b = check_3_planes_corr(plane_1, plane_2, new_plane)
            if new_check == []:
                Flag = True
                break
        if Flag == True:
            break
    return new_plane

def fix_4th_plane(plane_1,plane_2,plane_3,points_num,decay, sample, answer): 
    Flag = False
    for i in range(5, points_num, 3):
        for j in range(int(decay*1000),0,-1):
            new_decay = j/1000.0
            new_plane = find_4th_plane(sample, answer, plane_1, plane_2, plane_3, i, new_decay)
            new_check1,a1,b1 = check_3_planes_corr(plane_1, plane_2, new_plane)
            if (new_check1 == []):
                Flag = True
                break
        if Flag == True:
            break
    return new_plane

def fix_5th_plane(plane_1,plane_2,plane_3,plane_4,points_num,decay, sample, answer):
    for num in range(points_num_5,8,-3):
        for decay in range(5,int(decay_5*1000),5):
            decay /= 1000.0
            Plane = find_5th_plane(sample0, answer0, plane_1, plane_2, plane_3, plane_4, num, decay)
            plane5_equals = check_equals_planes(Plane,plane_1) | check_equals_planes(Plane,plane_2) | check_equals_planes(Plane,plane_3) | check_equals_planes(Plane,plane_4)
            if (check_3_planes_corr(Plane, plane_1, plane_2)[0] == []) and not plane5_equals:
                Flag = True
                break
        if Flag:
            break
    if (check_3_planes_corr(Plane, plane_1, plane_2)[0] != []) or plane5_equals:
        for num in range(points_num_5,40,3):
            for decay in range(1,int(decay_5*1000),5):
                decay /= 1000.0
                Plane = find_5th_plane(sample0, answer0, plane_1, plane_2, plane_3, plane_4, num, decay)
                plane5_equals = check_equals_planes(Plane,plane_1) | check_equals_planes(Plane,plane_2) | check_equals_planes(Plane,plane_3) | check_equals_planes(Plane,plane_4)
                if (check_3_planes_corr(Plane, plane1, plane2)[0] == []) and not plane5_equals:
                    Flag = True
                    break
            if Flag:
                break
    return Plane
    
def fix_1st_paral_plane(sample, answer,other_plane, paral_plane, points_num):
    main_box = find_cube_points(sample)
    main_box = main_box.set_index(np.arange(0, len(main_box)))

    new_box = delete_plane_dots(main_box,paral_plane,0.3)
    main_box = new_box
    new_box = delete_plane_dots(main_box,other_plane,0.3)
    main_box = new_box
    
    while (main_box.shape[0] > points_num):
        main_box = half_search(main_box)

    plane = find_plane(main_box)

    ans = answer.to_numpy()
        
    return plane

C:\Users\user\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\user\anaconda3\lib\site-packages\numpy\core\_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
C:\Users\user\anaconda3\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-30-e46ec678ec81>:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_box['class'][i] = 1
C:\Users\user\anaconda3\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\user\anaconda3

[-1.44992026 -0.47793812  0.36260155]
[-1.21374314  0.1390523   1.23938371]
[-1.44302565  0.41897376 -0.27041132]
[-1.20684854  1.03596418  0.60637084]
[-0.37093583  0.2772852  -0.45949203]
[-0.13475871  0.89427562  0.41729013]
[-0.37783043 -0.61962668  0.17352084]
[-0.14165332 -0.00263626  1.050303  ]
all points were found


<ipython-input-30-e46ec678ec81>:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_box['class'][i] = 1
<ipython-input-30-e46ec678ec81>:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_box['class'][i] = 1
<ipython-input-30-e46ec678ec81>:230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_box['class'][i] = 1


In [ ]:

sample0 = pd.read_csv('data\\sample0.csv', sep = ' ')
answer0 = pd.read_csv('data\\answer0.csv', sep = ' ')
ans = answer0.to_numpy()

radius = 0
for i in range(sample0.shape[0]):
    for j in range(sample0.shape[0]):
        x = [sample0['x'][i],sample0['y'][i],sample0['z'][i]]
        y = [sample0['x'][j],sample0['y'][j],sample0['z'][j]]
        if (dist_bt_points(x,y) > radius):
            radius = dist_bt_points(x,y)
max_side = round((radius*0.4 + 0.1)*10)/10


points_num_1 = 15
decay_1 = 0.15
plane1, center = find_1st_plane(sample0, answer0, points_num_1)

points_num_2 = 20
decay_2 = 0.15
plane2 = find_2nd_plane(sample0, answer0, plane1, points_num_2, decay_2)

points_num_3 = 15
decay_3 = 0.1
plane3 = find_3rd_plane(sample0, answer0, plane1, plane2, points_num_3, decay_3)


plane_info, paral_count, perpen_count = check_3_planes_corr(plane1, plane2, plane3) 

if plane_info == [1,2]:
    plane3 = fix_plane(plane1,plane2,points_num_3,decay_3, sample0, answer0)
if plane_info == [0,2]:
    plane2 = fix_plane(plane1,plane3,points_num_2,decay_2, sample0, answer0)
if plane_info == [0,1]:
    plane1 = fix_plane(plane2,plane3,points_num_1,decay_1, sample0, answer0)


have_some_equal = check_equals_planes(plane1,plane2) | check_equals_planes(plane1,plane3) | check_equals_planes(plane2,plane3) 
if (plane_info == []) and have_some_equal:
    points_paral_num = 10
    if check_equals_planes(plane1,plane2):
        while (check_equals_planes(plane1,plane2)) or (check_3_planes_corr(plane1, plane2, plane3)[0] != []):
            plane2 = fix_1st_paral_plane(sample0, answer0,plane3, plane1, points_paral_num)
            points_paral_num += 3
            if points_paral_num > 70:
                print("Broken")
                break
    if check_equals_planes(plane1,plane3):
        while (check_equals_planes(plane1,plane3)) or (check_3_planes_corr(plane1, plane2, plane3)[0] != []):
            plane3 = fix_1st_paral_plane(sample0, answer0,plane2, plane1, points_paral_num)
            points_paral_num += 3
            if points_paral_num > 70:
                print("Broken")
                break
    if check_equals_planes(plane2,plane3):
        while (check_equals_planes(plane2,plane3))  or (check_3_planes_corr(plane1, plane2, plane3)[0] != []):
            plane3 = fix_1st_paral_plane(sample0, answer0,plane1, plane3, points_paral_num)
            points_paral_num += 3
            if points_paral_num > 70:
                print("Broken")
                break


points_num_4 = 40
decay_4 = 0.05
plane4 = find_4th_plane(sample0, answer0, plane1, plane2, plane3, points_num_4, decay_4)


plane_info, paral_count, perpen_count = check_3_planes_corr(plane1, plane2, plane4)

plane4_equals = check_equals_planes(plane4,plane1) | check_equals_planes(plane4,plane2) | check_equals_planes(plane4,plane3)
while (plane_info != []) or plane4_equals:
    plane4 = fix_4th_plane(plane1,plane2,plane3,points_num_4,decay_4, sample0, answer0)
    plane_info, paral_count, perpen_count = check_3_planes_corr(plane1, plane2, plane4)
    plane4_equals = check_equals_planes(plane4,plane1) | check_equals_planes(plane4,plane2) | check_equals_planes(plane4,plane3)

plane_info, paral, perpen, index =  check_4_planes_corr(plane1, plane2, plane3, plane4)
#index - paralel planes: 0 - 1,2; 1 - 1,3; 2 - 1,4; 3 - 2,3; 4 - 2,4; 5 - 3,4


if paral == 2:
    points_num_5 = 20
    decay_5 = 0.05
    plane5 = find_5th_plane(sample0, answer0, plane1, plane2, plane3, plane4, points_num_5, decay_5)
    plane5_equals = check_equals_planes(plane5,plane1) | check_equals_planes(plane5,plane2) | check_equals_planes(plane5,plane3) | check_equals_planes(plane5,plane4) 
    Flag = False
    if (check_3_planes_corr(plane5, plane1, plane2)[0] != []) or plane5_equals:
        plane5 = fix_5th_plane(plane1,plane2,plane3,plane4,points_num_5,decay_5,sample0,answer0)


all_angles = []

if paral == 1:
    if index == 0:
        triple1 = [plane1,plane3,plane4]
        triple2 = [plane2,plane3,plane4]
    if index == 1:
        triple1 = [plane1,plane2,plane4]
        triple2 = [plane3,plane2,plane4]
    if index == 2:
        triple1 = [plane1,plane2,plane3]
        triple2 = [plane4,plane2,plane3]
    if index == 3:
        triple1 = [plane2,plane1,plane4]
        triple2 = [plane3,plane1,plane4]
    if index == 4:
        triple1 = [plane2,plane1,plane3]
        triple2 = [plane4,plane1,plane3]
    if index == 5:
        triple1 = [plane3,plane1,plane1]
        triple2 = [plane4,plane1,plane2]
        
    A = [triple1[0][0:3], triple1[1][0:3], triple1[2][0:3]]
    B = [triple1[0][3], triple1[1][3], triple1[2][3]]
    dot1 = np.linalg.solve(A, B)

    A = [triple2[0][0:3], triple2[1][0:3], triple2[2][0:3]]
    B = [triple2[0][3], triple2[1][3], triple2[2][3]]
    dot2 = np.linalg.solve(A, B)

    side = dist_bt_points(dot1,dot2)
    
    plane5 = build_paral_plane(triple1[1], side, center)
    plane6 = build_paral_plane(triple1[2], side, center)

    planes = [triple1[0],triple2[0],triple1[1],triple1[2],plane5,plane6]
    
    A = [planes[0][0:3], planes[2][0:3], planes[3][0:3]]
    B = [planes[0][3], planes[2][3], planes[3][3]]

    all_angles.append(np.linalg.solve(A, B))

    A = [planes[1][0:3], planes[2][0:3], planes[3][0:3]]
    B = [planes[1][3], planes[2][3], planes[3][3]]
    all_angles.append(np.linalg.solve(A, B))

    A = [planes[0][0:3], planes[3][0:3], planes[4][0:3]]
    B = [planes[0][3], planes[3][3], planes[4][3]]
    all_angles.append(np.linalg.solve(A, B))

    A = [planes[1][0:3], planes[3][0:3], planes[4][0:3]]
    B = [planes[1][3], planes[3][3], planes[4][3]]
    all_angles.append(np.linalg.solve(A, B))

    A = [planes[0][0:3], planes[4][0:3], planes[5][0:3]]
    B = [planes[0][3], planes[4][3], planes[5][3]]
    all_angles.append(np.linalg.solve(A, B))

    A = [planes[1][0:3], planes[4][0:3], planes[5][0:3]]
    B = [planes[1][3], planes[4][3], planes[5][3]]
    all_angles.append(np.linalg.solve(A, B))

    A = [planes[0][0:3], planes[5][0:3], planes[2][0:3]]
    B = [planes[0][3], planes[5][3], planes[2][3]]
    all_angles.append(np.linalg.solve(A, B))

    A = [planes[1][0:3], planes[5][0:3], planes[2][0:3]]
    B = [planes[1][3], planes[5][3], planes[2][3]]
    all_angles.append(np.linalg.solve(A, B))
    
#аналогично если было 2 пары паралельных
if paral == 2:
    if index == 0:
        paral_planes = [plane1,plane3,plane2,plane4]
    if index == 1:
        paral_planes = [plane1,plane2,plane3,plane4]
    if index == 2:
        paral_planes = [plane1,plane2,plane4,plane3]
    if index == 3:
        paral_planes = [plane1,plane2,plane4,plane3]
    if index == 4:
        paral_planes = [plane1,plane2,plane3,plane4]
    if index == 5:
        paral_planes = [plane1,plane3,plane2,plane4]    
    
    for i in range(4):
        A = [plane5[0:3],paral_planes[i][0:3],paral_planes[(i+1) % 4][0:3]]
        B = [plane5[3],paral_planes[i][3],paral_planes[(i+1) % 4][3]]
        all_angles.append(np.linalg.solve(A, B))
    
    side = dist_bt_points(all_angles[0],all_angles[1])
    
    plane6 = build_paral_plane(plane5, side, center)
    
    for i in range(4):
        A = [plane6[0:3],paral_planes[i][0:3],paral_planes[(i+1) % 4][0:3]]
        B = [plane6[3],paral_planes[i][3],paral_planes[(i+1) % 4][3]]
        all_angles.append(np.linalg.solve(A, B))
    #paralel planes: 0 - 1,2; 1 - 1,3; 2 - 1,4; 3 - 2,3; 4 - 2,4; 5 - 3,4
    

count = 0
for i in range(8):
    for j in range(8):
        if abs((all_angles[i][0] - ans[j][0] + all_angles[i][1] - ans[j][1] + all_angles[i][2] - ans[j][2])) < 1e-6:
            count += 1
    print(all_angles[i])
if count == 8:
    print('all points were found')
